[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Aleph-Alpha/examples/blob/main/exercises/08_exercise_g.ipynb)

# Exercise G: Create a Demo for Aleph Alpha Technology
Finally, in this notebook we will try to create a demo application with a small interface

This will help to assess how good a task can be completed.

In [ ]:
!pip install aleph_alpha_client gradio

from typing import Sequence
from aleph_alpha_client import ImagePrompt, AlephAlphaClient, AlephAlphaModel, SemanticEmbeddingRequest, SemanticRepresentation, Prompt, SummarizationRequest, CompletionRequest, EvaluationRequest, Document
import gradio as gr

import math
import os

In [ ]:
# instantiate the client and model
search_model = AlephAlphaModel.from_model_name("luminous-base", token="YOUR_TOKEN")

model = AlephAlphaModel.from_model_name("luminous-extended", token= "YOUR_TOKEN")

## Pre-defined functions
To make life a bit easier for you we have defined a few functions that you can use in this notebook.


They are described in this table:
|function|description|
|---|---|
|`embed_symmetric`| Embeds a text using the symmetric model|
|`embed_query`| Embeds a query using the asymmetric model|
|`embed_document`| Embeds a document using the asymmetric model|
|`cosine_similarity`| Calculates the cosine similarity between two vectors|
|`generate_summary`| Generates a summary of a document|
|`split_text`| Splits a text into paragraphs|
|`evaluate`| Uses the evaluate functionality of luminous to evaluate how good a completion fits to a query|

In [ ]:
# function for symmetric embeddings 
def embed_symmetric(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Symmetric)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Queries
def embed_query(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Query)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Documents
def embed_document(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document)
    result = search_model.semantic_embed(request)
    return result.embedding

# function to calculate similarity
def cosine_similarity(v1: Sequence[float], v2: Sequence[float]) -> float:
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

# function for getting a summary
def generate_summary(text: str):
    request = SummarizationRequest(document=Document.from_text(text))
    result = model.summarize(request)
    return result.summary

# function that splits text by paragraphs
def split_text(text: str):
    return text.split("\n\n")

# function that evaluate two texts
def evaluate(text1: str, text2: str):
    request = EvaluationRequest(prompt=Prompt.from_text(text1), completion_expected=text2)
    result = model.evaluate(request)
    return result

## Creating a small data extraction demo
We will create a small demo that extracts the most relevant information from emails. We will visualize the results in a small interface using gradio. 

The data we will extract is:
- The sender
- The receiver
- The intent of the email

In [ ]:
# TODO : write a prompt that extracts the sender and the receiver from the email.
# Keep in mind that the prompt should be able to handle multiple emails and that there may be no information about the sender or the receiver.

def extract_sender(text: str):
    prompt = """Write a prompt that extracts the sender from the email.""" #TODO
    
    request = CompletionRequest(prompt=Prompt.from_text(prompt), stop_sequences=["###"], max_tokens=20)
    completion = model.complete(request).completions[0].completion
    sender = completion
    return sender

def extract_receiver(text: str):
    prompt = """Write a prompt that extracts the receiver from the email.""" #TODO
    
    request = CompletionRequest(prompt=Prompt.from_text(prompt), stop_sequences=["###"], max_tokens=20)
    completion = model.complete(request).completions[0].completion
    receiver = completion
    return receiver

In [ ]:
# TODO : write a prompt that extracts the intent from the email.
def extract_intent(text: str):
    prompt = """Write a prompt that extracts the intent from the email.""" #TODO
    
    request = CompletionRequest(prompt=Prompt.from_text(prompt), stop_sequences=["###"], max_tokens=20)
    completion = model.complete(request).completions[0].completion
    intent = completion
    return intent

In [ ]:
# Small helper fuction that runs the above functions and returns the results
def run(text: str):
    sender = extract_sender(text)
    receiver = extract_receiver(text)
    intent = extract_intent(text)
    return sender, receiver, intent

## Visualizing the results
AI can be hard to understand. To make it easier for us to understand the results we will visualize them using gradio. Gradio is a library that allows you to create a small interface for your AI model. You can find more information about gradio [here](https://gradio.app/).

In [ ]:
# Gradio Interface that accepts a text and returns the sender, receiver and intent
interface = gr.Interface(
    fn=run,
    inputs=gr.components.Textbox(lines=10, placeholder="Enter your email here"),
    outputs=[
        gr.components.Textbox(label="Sender"),
        gr.components.Textbox(label="Receiver"),
        gr.components.Textbox(label="Intent"),
    ],
    title="Email Intent Extraction",
    description="This model extracts the sender, receiver and intent from an email."
)
interface.launch()